# **AlexNet**

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import zipfile
import os

# Unzip only if not already extracted
zip_path = r"C:\Users\Admin\Downloads\cracked.zip"
extract_path = "crack_dataset"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
train_dir = os.path.join("crack_dataset", "train")
val_dir = os.path.join("crack_dataset", "val")

In [ ]:
data_gen= ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)
train_generator = data_gen.flow_from_directory(
    extract_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training')
val_gen = data_gen.flow_from_directory(
    extract_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation')

In [ ]:
# Define the AlexNet architecture
model_alexnet = tf.keras.Sequential([
    # Layer 1
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((3, 3), strides=(2, 2)),
    # Layer 2
    Conv2D(256, (5, 5), activation='relu', padding='same'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    # Layer 3
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    # Layer 4
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    # Layer 5
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    Flatten(),
    # Layer 6
    Dense(4096, activation='relu'),
    Dropout(0.5),
    # Layer 7
    Dense(4096, activation='relu'),
    Dropout(0.5),
    # Output layer
    Dense(1, activation='sigmoid')
])

# Summary of the model
model_alexnet.summary()

In [ ]:
model_alexnet.compile(
     optimizer=Adam(learning_rate=1e-4),
     loss='binary_crossentropy',
     metrics=['accuracy'])
# Define the EarlyStopping callback
early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
history=model_alexnet.fit(
     train_generator,
     validation_data=val_gen,
     epochs=3,
     callbacks=[early],
     verbose=1)

# **Saving the model**

In [ ]:
model_alexnet.save('model_alexnet.h5')
# Save history
import pickle
with open('history.pkl',"wb") as f:
    pickle.dump(history.history,f)

# **Load the Model Model**

In [ ]:
loaded_model = load_model('model_alexnet.h5')
with open('history.pkl',"rb") as f:
    history=pickle.load(f)

# **Visualization of training over epoch**

In [ ]:
plt.plot(history['accuracy'], label='train_accuracy')
plt.plot(history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# **Prediction**

In [ ]:
class_labels=["Cracked", "Not Cracked"]
def predict_crack(img_path):
    img=image.load_img(img_path,target_size=(224,224))
    img_array=image.img_to_array(img)
    img_array=img_array/255.0
    img_array=np.expand_dims(img_array,axis=0)
    prediction=loaded_model.predict(img_array)
    predicted_class=np.argmax(prediction,axis=1)
    confidence=np.max(prediction)
    print(f"Prediction: {class_labels[predicted_class]} (Confidence: {confidence:.2f})")
    plt.imshow(img)
    plt.title(f"{class_labels[predicted_class]} {confidence*100:.2f}")
    plt.axis("off")
    plt.show()
test_path=input("Enter image path: ")
predict_crack(test_path)